In [23]:
import pandas as pd
from datetime import datetime
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

df = pd.read_csv("data/raw_partner_headlines.csv")


In [2]:
# Grundlegende Informationen über das Dataset
print("Grundlegende Informationen über das Dataset:")
df_info = df.info()
print(df_info)

Grundlegende Informationen über das Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845559 entries, 0 to 1845558
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   headline    object
 2   url         object
 3   publisher   object
 4   date        object
 5   stock       object
dtypes: int64(1), object(5)
memory usage: 49.3+ MB
None


In [3]:
# Erster Blick auf die Daten
print("\nErster Blick auf die Daten:")
df_head = df.head()
print(df_head)


Erster Blick auf die Daten:
   Unnamed: 0                                           headline  \
0           2  Agilent Technologies Announces Pricing of $5……...   
1           3  Agilent (A) Gears Up for Q2 Earnings: What's i...   
2           4  J.P. Morgan Asset Management Announces Liquida...   
3           5  Pershing Square Capital Management, L.P. Buys ...   
4           6  Agilent Awards Trilogy Sciences with a Golden ...   

                                                 url  publisher  \
0  http://www.gurufocus.com/news/1153187/agilent-...  GuruFocus   
1  http://www.zacks.com/stock/news/931205/agilent...      Zacks   
2  http://www.gurufocus.com/news/1138923/jp-morga...  GuruFocus   
3  http://www.gurufocus.com/news/1138704/pershing...  GuruFocus   
4  http://www.gurufocus.com/news/1134012/agilent-...  GuruFocus   

                  date stock  
0  2020-06-01 00:00:00     A  
1  2020-05-18 00:00:00     A  
2  2020-05-15 00:00:00     A  
3  2020-05-15 00:00:00     A  
4  2

In [14]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [16]:
df.drop(columns=['url'], inplace=True)
df.drop(columns=['publisher'], inplace=True)



In [17]:
# Statistische Zusammenfassung
print("\nStatistische Zusammenfassung:")
df_describe = df.describe(include='all')
print(df_describe)


Statistische Zusammenfassung:


C:\Users\youne\AppData\Local\Temp\ipykernel_2052\706450308.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_describe = df.describe(include='all')


                       headline                 date    stock  quarter
count                   1845559              1845559  1845559  1845559
unique                   806998                 3731     6552       43
top     Midday Gainers / Losers  2015-11-19 00:00:00       KR   2019Q4
freq                       4009                 3600     3314    98474
first                       NaN  1969-12-31 00:00:00      NaN      NaN
last                        NaN  2020-06-04 00:00:00      NaN      NaN


In [18]:
# Überprüfung auf fehlende Werte
print("\nÜberprüfung auf fehlende Werte:")
df_missing_values = df.isnull().sum()
print(df_missing_values)


Überprüfung auf fehlende Werte:
headline    0
date        0
stock       0
quarter     0
dtype: int64


In [7]:
# Verteilung der Aktien
print("\nVerteilung der Aktien:")
df_stock_distribution = df['stock'].value_counts()
print(df_stock_distribution)


Verteilung der Aktien:
KR       3314
GXC      3238
PGJ      3082
YINN     3027
JPM      2873
         ... 
ARCPP       1
HGH         1
GFNSL       1
DIA         1
JBK         1
Name: stock, Length: 6552, dtype: int64


In [19]:
# Verteilung der Daten über die Zeit
print("\nVerteilung der Daten über die Zeit:")
df['date'] = pd.to_datetime(df['date'])
df_date_distribution = df['date'].dt.to_period('M').value_counts().sort_index()
print(df_date_distribution)


Verteilung der Daten über die Zeit:
1969-12        1
2010-02      367
2010-03     1700
2010-04     1505
2010-05     1120
           ...  
2020-02    24532
2020-03    16077
2020-04    20566
2020-05    18832
2020-06     1193
Freq: M, Name: date, Length: 126, dtype: int64


In [9]:
# Datentyp der 'date'-Spalte konvertieren
df['date'] = pd.to_datetime(df['date'])

# Verteilung der Daten über die Quartale
df['quarter'] = df['date'].dt.to_period('Q')
df_quarter_distribution = df['quarter'].value_counts().sort_index()

In [33]:
# Sicherstellen, dass die benötigten NLTK-Resourcen heruntergeladen sind
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\youne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\youne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\youne\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\youne\AppData\Roaming\nltk_data...


True

In [20]:
#Func zur Testbereinigung 
def clean_text(text):
    text = re.sub(r'<.*?>', '', text) # del von HTML-Tags
    text = re.sub(r'[^a-zA-Z\s]', '', text) # del von Sonderzeichen
    text = text.lower()
    return text



In [28]:
# Funktion zur Texttokenisierung, Entfernen von Stoppwörtern und Lemmatisierung

def preprocess_text(text):
    # Tokenisierung
    tokens = word_tokenize(text)
    # Entfernen von Stoppwörtern
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatisierung
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Zusammenfügen der Tokens zu einem String
    processed_text = ' '.join(tokens)
    
    return processed_text

